In [1]:
import json
from pprint import pprint
from os import listdir
from os.path import isfile, join

In [2]:
dump = {}
for fname in listdir('../db/'):
    with open('../db/' + fname, 'r') as f:
        dump[fname.split('.')[0]] = json.load(f)

In [3]:
sorted(list(dump.keys()))

['drones',
 'equipmentTypes',
 'equipments',
 'gamestarts',
 'missileSpecOpts',
 'missileSpecs',
 'missileTypes',
 'missiles',
 'objMain',
 'objSubtypeShip',
 'objSubtypeStation',
 'productTypes',
 'products',
 'races',
 'sectorAteroids',
 'sectorDebries',
 'sectorGateTypes',
 'sectorGates',
 'sectorShips',
 'sectorShipyardShips',
 'sectorShipyardStations',
 'sectorSpaceportEquipments',
 'sectorSpaceportProducts',
 'sectorStations',
 'sectors',
 'shieldTypes',
 'shields',
 'shipClasses',
 'shipCockpitWeapon',
 'shipCockpits',
 'shipHangars',
 'shipMissileGroups',
 'shipMissiles',
 'shipProduction',
 'ships',
 'stationClasses',
 'stationProducts',
 'stationResourceSecondaries',
 'stationResources',
 'stationTypes',
 'versions',
 'weaponSpecDatas',
 'weaponSpecs',
 'weaponTypes',
 'weapons']

In [4]:
def tc(version):
    return version <= 35

def ap(version):
    return version >= 35

In [5]:
def v(version):
    return tc(version)

In [6]:
def identitize(key, index='id'):
    if 'version' in dump[key][1]:
        return {x[index]: x for x in dump[key] if v(x['version'])}
    return {x[index]: x for x in dump[key]}

## Products 

In [7]:
rawProducts = identitize('products')

In [8]:
rawProductTypes = identitize('productTypes')

In [9]:
def fixProducts(i):
    product = rawProducts[i].copy()
    product['type'] = rawProductTypes[product['idType']].copy()
    product.pop('idType', None)
    product['type'].pop('id', None)
    
    return product

In [10]:
products = {p: fixProducts(p) for p in rawProducts}

In [11]:
pprint(products[1])

{'cargo': 1,
 'crMax': 20,
 'crMed': 16,
 'crMin': 12,
 'descr': 'Le Celle di energia sono l&#39;unit&agrave; di accumulo di energia '
          'usata in tutto l&#39;universo conosciuto. L&#39;unificazione delle '
          'unit&agrave; di energia permette il commercio interstellare di '
          'quasi tutti i prodotti di base.',
 'id': 1,
 'idGame': 'SS_WARE_ENERGY',
 'name': 'Celle di Energia',
 'type': {'descr': "Fonti di energia attualmente in uso nell'universo.",
          'name': 'Energia'}}


## Weapons

In [12]:
rawWeapons = identitize('weapons')

In [13]:
rawWeaponTypes = identitize('weaponTypes')

In [14]:
rawWeaponSpecs = identitize('weaponSpecs')

In [15]:
rawWeaponSpecDatas = [x for x in dump['weaponSpecDatas'] if v(x['version'])]

In [16]:
def fixWeapons(i):
    weapon = rawWeapons[i].copy()
    weapon['product'] = products[weapon['idProduct']]
    weapon['type'] = rawWeaponTypes[weapon['idType']].copy()
    weapon['specs'] = []
    for x in rawWeaponSpecDatas:
        if x['id'] == weapon['id']:
            spec = rawWeaponSpecs[x['idSpec']].copy()
            spec.pop('id', None)
            weapon['specs'].append(spec)
    
    weapon.pop('idType', None)
    weapon['type'].pop('id', None)
    weapon.pop('idSpec', None)
    weapon.pop('idProduct', None)
    
    return weapon

In [17]:
weapons = {w: fixWeapons(w) for w in rawWeapons}

In [18]:
weapons[8]

{'ammo': 0,
 'dmgHull': 0.0,
 'dmgShield': 0.0,
 'enMin': 7000,
 'enShot': 1000,
 'id': 8,
 'oosHull': 0.0,
 'oosShield': 0.0,
 'product': {'cargo': 50,
  'crMax': 794669,
  'crMed': 779088,
  'crMin': 763507,
  'descr': 'Il Raggio Traente viene usato per spostare oggetti nello spazio.<br/><br/>Incredibile come un dispositivo cos&igrave; complesso abbia una descrizione cos&igrave; semplice, vero?',
  'id': 56,
  'idGame': 'SS_LASER_TUG',
  'name': 'Raggio Traente',
  'type': {'descr': 'Armi usate nelle astronavi.', 'name': 'Armi'}},
 'range': 2.0,
 'rate': 120.0,
 'specs': [{'descr': "L'arma produce un fascio di particelle continuo.",
   'spec': 'Arma a raggio'}],
 'speed': 4000.0,
 'type': {'descr': 'Armi inoffensive utilizzate per operazioni di carattere civile.',
  'name': 'Civile'},
 'version': 35}

## Missiles

In [19]:
rawMissiles = identitize('missiles')

In [20]:
rawMissileTypes = identitize('missileTypes')

In [21]:
rawMissileSpecOpts = identitize('missileSpecOpts')

In [22]:
rawMissileSpecs = [x for x in dump['missileSpecs'] if v(x['version'])]

In [23]:
def fixMissiles(i):
    missile = rawMissiles[i].copy()
    missile['product'] = products[missile['idProduct']]
    missile['type'] = rawMissileTypes[missile['idType']].copy()
    missile['specs'] = []
    for x in rawMissileSpecs:
        if x['id'] == missile['id'] and v(x['version']):
            opt = rawMissileSpecOpts[x['idSpec']].copy()
            opt.pop('id', None)
            missile['specs'].append(opt)
    
    missile.pop('idType', None)
    missile['type'].pop('id', None)
    missile.pop('idSpec', None)
    missile.pop('idProduct', None)
    
    return missile

In [24]:
missiles = {m: fixMissiles(m) for m in rawMissiles}

In [25]:
missiles[1]

{'accel': 132.0,
 'area': 150.0,
 'dmg': 300000.0,
 'fuel': 300.0,
 'id': 1,
 'product': {'cargo': 20,
  'crMax': 54073,
  'crMed': 50536,
  'crMin': 46999,
  'descr': 'Questo missile &egrave; stato progettato come grande torpedine anti-capital e pu&ograve; essere usato con un lancio a grappolo specialmente dalle fregate missilistiche. L&#39;Hammer Heavy Torpedo combina una grande testata con una discreta velocit&agrave; e una buona manovrabilit&agrave;. Queste caratteristiche sono rese possibili grazie ad un sistema di lancio creato appositamente ed installato su una classe di nave appositamente creata per il lancio di missili.',
  'id': 109,
  'idGame': 'SS_MISSILE_TORP_CAPITAL',
  'name': 'Hammer Heavy Torpedo',
  'type': {'descr': 'Proiettili propellenti spesso auto-guidati urilizzati dalle navi.',
   'name': 'Missili'}},
 'range': 75.9,
 'rate': 0.1,
 'reload': 1.25,
 'specs': [{'descr': 'Il missile detona in prossimitÃ\xa0Â\xa0 di un bersaglio.',
   'name': 'Innesco di prossimitÃ

## Shields

In [26]:
rawShields = identitize('shields')

In [27]:
rawShieldTypes = identitize('shieldTypes')

In [28]:
def fixShields(i):
    shield = rawShields[i].copy()
    shield['product'] = products[shield['idProduct']]
    shield['type'] = rawShieldTypes[shield['idType']].copy()
    
    shield.pop('idType', None)
    shield['type'].pop('id', None)
    shield.pop('idProduct', None)
    
    return shield

In [29]:
shields = {s: fixShields(s) for s in rawShields}

In [30]:
shields[1]

{'enRechanrge': 33,
 'energy': 1,
 'id': 1,
 'product': {'cargo': 1,
  'crMax': 6230,
  'crMed': 5192,
  'crMin': 4154,
  'descr': 'Questo scudo &egrave; uno dei primissimi dispositivi di difesa inventati nel corso della storia. Originariamente era pensato come semplice deflettore per detriti spaziali e micro meteoriti. Attualmente questo scudo viene utilizzato su piccoli caccia, ricognitori e alcuni droni.',
  'id': 36,
  'idGame': 'SS_SHIELD_A',
  'name': 'Scudo da 1MJ',
  'type': {'descr': 'Scudi energetici usati sulle navi.', 'name': 'Scudi'}},
 'rechanrgeRate': 33,
 'rechargeTime': 30,
 'type': {'descr': 'Scudi utilizzati da piccole navi e droni.',
  'name': 'Leggero'}}

## Ships

In [31]:
rawClasses = identitize('shipClasses')

In [32]:
rawRaces = identitize('races')

In [33]:
rawShips = identitize('ships')

In [34]:
rawShipMissileGroups = {}

for mg in dump['shipMissileGroups']:
    if v(mg['version']):
        if mg['id'] not in rawShipMissileGroups:
            rawShipMissileGroups[mg['id']] = {'idGroup': mg['id'], 'missiles': []}
        m = missiles[mg['idMissile']].copy()
        rawShipMissileGroups[mg['id']]['missiles'].append(m)

In [35]:
rawShipMissiles = {}

for mg in dump['shipMissiles']:
    if v(mg['version']):
        if mg['idShip'] not in rawShipMissiles:
            rawShipMissiles[mg['idShip']] = []
        m = rawShipMissileGroups[mg['idGroup']].copy()
        rawShipMissiles[mg['idShip']].append(m)

In [41]:
rawShipCockpitWeapon = {}

for c in dump['shipCockpitWeapon']:
    if v(c['version']):
        idc = c['idCockpit']
        if idc not in rawShipCockpitWeapon:
            rawShipCockpitWeapon[idc] = []
        w = weapons[c['idWeapon']].copy()
        rawShipCockpitWeapon[idc].append(w)

In [42]:
rawShipCockpits = {}

for ck in dump['shipCockpits']:
    c = ck.copy()
    if v(c['version']):
        ids = c['idShip']
        if ids not in rawShipCockpits:
            rawShipCockpits[ids] = []
        c['weapons'] = rawShipCockpitWeapon[c['idCockpit']]
        c.pop('id', None)
        c.pop('idShip', None)
        c.pop('version', None)
        rawShipCockpits[ids].append(c)

In [46]:
rawShipProduction = identitize('shipProduction', 'idShip')

In [47]:
rawShipHangar = {}

for sh in dump['shipHangars']:
    if v(sh['version']):
        if sh['idShip'] not in rawShipHangar:
            rawShipHangar[sh['idShip']] = []
        rawShipHangar[sh['idShip']].append(sh)

In [48]:
def fixShip(i):
    ship = rawShips[i].copy()
    ship['race'] = rawRaces[ship['idRace']].copy()
    ship['class'] = rawClasses[ship['idClass']].copy()
    ship['shields'] = shields[ship['shieldsType']].copy()
    ship['production'] = rawShipProduction[ship['id']].copy()
    ship['hangarType'] = []
    
    if ship['hangar'] is None:
        ship['hangar'] = 0

    if ship['hangar'] > 0:
        for h in rawShipHangar[ship['id']]:
            c = rawClasses[h['idClass']].copy()
            c.pop('id', None)
            ship['hangarType'].append(c)
    
    ship['missiles'] = rawShipMissiles[ship['id']].copy()
    ship['weapons'] = rawShipCockpits[ship['id']].copy()
    
    ship.pop('idRace', None)
    ship.pop('idClass', None)
    ship.pop('shieldsType', None)
    ship.pop('id')
    ship['race'].pop('id', None)
    ship['class'].pop('id', None)

    return ship

In [49]:
pprint(fixShip(41))

{'accelMax': 11.6,
 'accelMin': 6.8,
 'cargoMax': 4750,
 'cargoMin': 2500,
 'cargoType': 'XL',
 'class': {'descr': 'Fregata missilistica',
           'idGame': 'SG_SH_M7',
           'name': 'M7M'},
 'crMax': 38206712,
 'crMin': 0,
 'descr': 'Nave sorella dell&#39;<a '
          'href=&quot;navi.php?race=Argon&amp;class=9&amp;id_ship=39&quot;>OTAS '
          'Aquilo</a>, questa nave &egrave; pensata unicamente per '
          'l&#39;abbordaggio grazie alla capacit&egrave; di trasportare fino a '
          'trenta marine. La totale assenza di un armamento offensivo, ad '
          'eccezione di una torretta posteriore, la rende inutile fuori dal '
          'suo contesto: gli abbordaggi.<br/><br/><span style=&#39;color: '
          'orange&#39;>Questa nave &egrave; disponibile solo per gli affiliati '
          'alla Corporazione OTAS.</span>',
 'hangar': 0,
 'hangarType': [],
 'hull': 99000,
 'missiles': [{'idGroup': 'BOARDINGPOD',
               'missiles': [{'accel': 40.0,
         